# Introduction

In this notebook we will:
* Get to analyze life expectancy data gathere from World Health Organization
* Learn the main steps of the Exploratory Data Analysis (EDA)
* Discuss different techniques of data preprocessing 

Please leave feedback after each seminar [here](https://forms.gle/sLB8NrZZoaWgYu6F7).

## Dataset

Today we will work with life expectancy data gathered from the World Health Organization (WHO) website. Our goal is to perform exploratory data analysis, preprocess the dataset, and use linear regression to understand the factors associated with increased life expectancy.

Original dataset is hosted on the [Kaggle](https://www.kaggle.com/kumarajarshi/life-expectancy-who) platform. To avoid the hassle of logging in, we'll use a file generously posted on GitHub by a random internet user:

In [ ]:
# Use wget console utility to download the csv data
!wget "https://gist.githubusercontent.com/aishwarya8615/89d9f36fc014dea62487f7347864d16a/raw/8629d284e13976dcb13bb0b27043224b9266fffa/Life_Expectancy_Data.csv"

In [ ]:
# Make sure that we actually downloaded Life_Expectancy_Data.csv file
!ls

# Exploratory data analysis (EDA)

In short, EDA is a vaguely defined term that describes the process of “getting to know data”. Often includes preprocessing, visualization, and data summarization.

We'll start from scratch and first load our dataset and get a list of the available features.

In [ ]:
import pandas as pd

# load the dataset (df stands for the DataFrame)
df = pd.read_csv("Life_Expectancy_Data.csv")

In [ ]:
# print the 'shape' of the matrix
print(f"Rows x columns: {df.shape}")

In [ ]:
# print the first 4 lines
...

There are 23 features and 2461 entries in total. Our target variable is called `Life_expectancy`.


## Preprocessing

Always rename columns if they contain unexpected spaces, forward slashes (/) or follow awkward naming schemes.

In [ ]:
# Dictionary(mapping), old name -> new name
mapping = {
  'Life_expectancy ': 'Life expectancy',
  'Adult_Mortality': 'Adult mortality',
  'infant_deaths': 'Infant deaths',
  'percentage_expenditure': 'Percentage expenditure',
  'Hepatitis_B': 'Hepatitis B',
  'Measles ': 'Measles',
  ' BMI ': 'BMI',
  'under_five_deaths ': 'Under-five deaths',
  'Total_expenditure': 'Total expenditure',
  'Diphtheria ': 'Diphtheria',
  ' HIV/AIDS': 'HIV/AIDS',
  ' thinness  1-19 years': 'Thinnes (1-19 years)',
  ' thinness 5-9 years': 'Thinnes (5-9 years)',
  'Income_composition_of_resources': 'Income composition of resources'
}
# Rename columns
...

In [ ]:
# Check the tail (or head)
...

Let's check if there are any missing values.

In [ ]:
# Special function to print inferred data types for all columns,
# whether or not they contain null (missing) values
...

Unfortunately, the dataset is not complete — several columns have missing values. How many nulls are there in each column?

In [ ]:
# Calculate total number of nulls for each column:

# 1. isnull() returns a mask, where null values is replaced with True
isnull = ...
# 2. Count null records for each columns
# (true boolean values are interpretted as 1)
isnull = ...
# 3. print columns with 1 or more null value
...

Let's discuss the following questions:

* *What can we do with null values?*
* *Can you imagine situations where one strategy is preferred over the other one?*
* *What strategies are preffered for large/small datasets?*

Let's try a few examples of how to drop null values.

In [ ]:
df[...]

In [ ]:
df.dropna()

In [ ]:
print(f"Rows before: {df.shape[0]}")

# Drop null rows(records)

# 1. Again, matrix with True in place of nulls
isnull = ...
# 2. Logical OR row-wise(axis=1). Are there ANY true value in each row?
isnull = ...
# sum true values = number of rows with nulls
print(f"\tWill be dropped {...} rows")
# 3. Select only rows without null values
notnull = ~... # ~ stands for element-wise NOT
df = df[notnull]

print(f"Rows after: {df.shape[0]}")

# Same as above, but using builtin function
# df.dropna(axis='index', how='any', inplace=True) # index = rows

In [ ]:
df = df.copy()

Typically, pandas can automatically detect the datatype for columns with only numeric or date records.

However, you will often come across datasets in which NULL values ​​are replaced with strings such as "Unknown", "NA". Another example is categorical columns built from strings.

In these cases, pandas uses the dtype `object` to efficiently store the full variety of data.

Here is an example for our dataset:

In [ ]:
# dtypes stands for Data Types
df.dtypes

In most cases, you want to get rid of `object` columns by converting them to categorical or numeric dtypes. It eases the analysis and eliminates silly errors that might arise during future analysis.

In practice, this is achieved by careful column-to-column analysis of the data:

In [ ]:
# Print all unique values
print("Unique continents:", df['Continent'].unique())
# No unusuall entries -> cast column to categorical data type
df['Continent'] = df['Continent'].astype('category')

print(f"New continent data type: {df['Continent'].dtype}")

In [ ]:
# Similarly, cast Country and Status to categorical
for x in 'Status', 'Country':
  print(f"{x} values: {df[x].unique()}\n")
  df[x] = df[x].astype('category')

In [ ]:
# The new categorical columns include a .cat property with a .categories nested
# property that lists all the available categories for the column:
print("Countries: ", df['Country'].cat.categories)

Another useful function for dealing with unknown data is `df[column].value_counts()`:

In [ ]:
df['Population'].value_counts()

In [ ]:
assert df['Population'].isna().sum() == 0
# Skip records with unknown population
mask = df['Population'] != 'Unknown'
df = df[mask].copy()

# Cast column to float data type
df['Population'] = df['Population'].astype(float)

In [ ]:
assert df['Diphtheria'].isna().sum() == 0
df['Diphtheria'] = df['Diphtheria'].astype(int)

Make sure we no longer have `object` columns:

In [ ]:
df.info()

That's all for now! But is that all we need before moving on to visualization?...

In fact, the following table is a key part of "preprocessing" in a broad sense.
You **must** understand your data, know what the numeric values ​​in each column mean, before going any further.

|Field|Description|
|---:|:---|
|Life expectancy|Life Expectancy in age|
|Adult Mortality|Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)|
|infant deaths|Number of Infant Deaths per 1000 population|
|Alcohol|Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)|
|Percentage expenditure|Expenditure on health as a percene of Gross Domestic Product per capita(%)|
|Hepatitis B|Hepatitis B (HepB) immunization coverage among 1-year-olds (%)|
|Measles|Measles - number of reported cases per 1000 population|
|BMI|Average Body Mass Index of entire population|
|Under-five deaths|Number of under-five deaths per 1000 population|
|Polio|Polio (Pol3) immunization coverage among 1-year-olds (%)|
|Total expenditure|General government expenditure on health as a percene of total government expenditure (%)|
|Diphtheria|Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)|
|HIV/AIDS|Deaths per 1 000 live births HIV/AIDS (0-4 years)|
|GDP|Gross Domestic Product per capita (in USD)|
|Population|Population of the country|
|Thinness (1-19 years)|Prevalence of thinness among children and adolescents for Age 10 to 19 (%)|
|Thinness (5-9 years)|Prevalence of thinness among children for Age 5 to 9(%)|
|Income composition of resources|Income composition of resources|
|Schooling|Number of years of Schooling(years)|

## Visualization

We will start our visualization with simple box plots, which usually have the following notation:

<center>
<img src="https://previews.dropbox.com/p/thumb/ACCcevd-ZKxEWIWALqW4Y2oSG0-lv5e925hceCNr6eH7x8tG1-HbscaJ7CRTpHCr_CvlZhXeKTPcdgn-umlmKpGCCGEGOLfcp6Sph7lm5BjBs7Ap7znhFNI6u4j4C6dtjKBP2IadVHuhjcD4ONc-CTnF_vHiaKitor4ewIK5rkvK7JWJmrXdp_7RhTB9jKQD9fX_MiBOnt3xYHmtpYi8Ntr7tI-jag-ZhCIEaRnsqyQul55kZ7lxXqVHFk1E9CO6LqDOxkF7I6qT1fRE5clm8doZLrQltGHyIpU119Ps5qrGRfCSeCDDW494NRP2qIr3l2PFgnC0iro4wSG98Zxq8-Jb/p.png" width=550>
</center>



In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Get numeric columns only
numeric = list(df.select_dtypes(include=np.number).columns)

# Create figure with 20 subplots
fig, axes = plt.subplots(..., figsize=(26, 14))
# By default, axes is a 4x5 matrix, ravel it to a flat array with 20 elements
axes = axes.ravel()

# Sanity check
assert len(numeric) == len(axes)

# Plot boxplot for each numeric column
for col, ax in zip(numeric, axes):
  sns.boxplot(y = df[col], ax=ax)
  #sns.violinplot(y = df[col], ax=ax)
  ax.set_title(col)
  ax.set(xlabel='', ylabel='')

`describe` method is a nice complementary way to better understand the composition of each column:

In [ ]:
# Perform `describe` only for numeric columns
df[numeric].describe()

We can also create a histogram for each column using the built-in pandas method:

In [ ]:
# `_ =` construct to ignore return data from the hist method
_ = df[numeric].hist(figsize=(22, 18), bins=25)

Discussion time!

* *Do we have columns that look extremely suspicious in general?*
* *Are there "special" values ​​in any of the numeric columns?*
* *What can we do with the outliers?*

One way to deal with outliers is to "trim" the data and keep only records where values in each column are within [$\alpha$, 1-$\alpha$] percentiles.

In [ ]:
print(f"Records before: {len(df)}")

# Get 0.01 and 0.99 percentile for each column, alpha = 0.01
low, high = 0.01, 0.99
quantiles = df[numeric].quantile([low, high])

# Remove Year column from the 'trimming' procedure
quantiles.drop(columns=['Year'], inplace=True)

for col in quantiles.columns:
  low, high = quantiles[col].tolist()
  mask = (...) & (...)
  df = df[mask]

print(f"Records after: {len(df)}")


In [ ]:
quantiles

In [ ]:
_ = df[numeric].hist(figsize=(22, 18), bins=25)

So, we dropped about 300 records, but there is still a suspicious peak at '0' for the income composition of resources...

*Why?*

In [ ]:
print(f"Recods before: {len(df)}")
df = df[df['Income composition of resources'] > 1e-6].copy()
print(f"Recods after: {len(df)}")

In [ ]:
df['Income composition of resources'].hist(bins=25)

Okay, all the fuss is interesting, but let's get down to business. We are (presumably) paid to interpret, not clean up the data(even if it is a part of the process).

Building a correlation heatmap is a good place to start:

In [ ]:
correlations = df[numeric].corr(method='spearman') # try pearson

plt.figure(figsize=(15,15))
sns.heatmap(correlations, square=True, annot=True, linewidths=0.25)
plt.title("Correlation matrix for numeric features")
plt.show()

* *What expected / unexpected correlations can one find here?*
* *Can you explain them with common sense?*

Never forget that correlation does not imply causation. [Here](https://www.tylervigen.com/spurious-correlations) are a few funny examples.

Can you explain them with common sense?

To keep you busy a little longer, here is the relationship between longevity and each feature:

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(26, 14))
axes = axes.ravel()

for ax, col in zip(axes, numeric):
  ax.scatter(df[col], df['Life expectancy'])
  ax.set_title(col)

What does it take to confidently argue that overall life expectancy increased from 2000 to 2015?

In other words, imagine that you have to defend this point of view during a WHO meeting. What tools/graphs would you use?

The game, find your country, has begun:

In [ ]:
countries = df.groupby('Country')['Life expectancy'].mean()

ax = countries.plot(kind='bar', figsize=(50,15), fontsize=25)
ax.set_title("Life_Expectancy w.r.t Country",fontsize=40)
ax.grid()

Empty columns indicate categories that were present in the original dataset, but disappeared after filtering.

Let's remove these "empty" categories from the general list:

In [ ]:
# select only categorical columns
categorical = df.select_dtypes('category').columns.tolist()

for col in categorical:
  # remove unused categories inplace(!)
  df[col].cat.remove_unused_categories()
  # how to run this without inplace?

* How do we perform EDA if given dataset has too many features?

# ML-specific preprocessing

## Dummy variables (one-hot encoding)

Dummy variables are a way to encode categorical data in numeric format.

This is commonly used for machine learning since fundamental math models cannot handle strings / categories by design (in most cases).

Overall, we have 20 columns with 1 target variable.
Among them, we will drop 'Country', 'Continent' and 'Year' features. *Why?*

In [ ]:
# Glue list with columns
columns = numeric + categorical
# Drop columns in-place
for x in "Country", "Continent", "Year":
  columns.remove(x)
# Select only remaining columns from the data frame
df = df[columns]

In [ ]:
# Replace column Status with dummy variables
ddf = pd.get_dummies(df, columns=['Status'])

In [ ]:
ddf

Dummy variables (also known as hot-coded functions) are constructed from categorical data as follows:
1. Add N new columns, where N is the number of categories.
2. For each entry, set 0 for all N categories except one corresponding to the current row category.


*How would you one-hot encode DNA sequences?*

## Train/test split

*Why random_state is fixed in the next block of code?*

In [ ]:
from sklearn.model_selection import train_test_split

Y = ddf.pop("Life expectancy")
Y = Y.values
X = ddf.values

print("Before:")
print(f"\tX: {X.shape}; Y: {Y.shape}")


X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state = 42
)

print("Train:")
print(f"\tX: {X_train.shape}; Y: {Y_train.shape}")
print("Test:")
print(f"\tX: {X_test.shape}; Y: {Y_test.shape}")


## Data Transformation

* Machine learning models make a lot of assumptions about the data

* In reality, these assumptions are often violated

* We build pipelines that transform the data before feeding it to the learners

    * Scaling (or other numeric transformations)

    * Encoding (convert categorical features into numerical ones)

    * Automatic feature selection

    * Feature engineering (e.g. binning, polynomial features,…)

    * Handling missing data

    * Handling imbalanced data

    * Dimensionality reduction (e.g. PCA)

    * Learned embeddings (e.g. for text)

* Seek the best combinations of transformations and learning methods

    * Often done empirically, using cross-validation

    * Make sure that there is no data leakage during this process!

Checkout [this page](https://ml-course.github.io/master/notebooks/06%20-%20Data%20Preprocessing.html#lecture-6-data-preprocessing) for more info and examples for data preprocessing tricks.

### Normalization

Most machine learning algorithms perform better with normalized data, i.e. ***approximately*** normal. For example, this is important for PCA and gradient methods, but not for decision trees.

The only trick here is that the normalization parameters should be determined only on the training set. We also don't really care about categorical columns, they will be processed together with numerical columns.

* *Why?*

* Makes sure that feature values of each point (each row) sum up to 1 (L1 norm)

    * Useful for count data (e.g. word counts in documents)

* Can also be used with L2 norm (sum of squares is 1)

    * Useful when computing distances in high dimensions

    * Normalized Euclidean distance is equivalent to cosine similarity

### Power transform

The first step is to remove skewness for features like measles and population. We will do this by a so-called [power transform](https://en.wikipedia.org/wiki/Power_transform).

Another option is the logarithmic transformation (e.g. $\hat{x} = log(x + 1)$), which is especially popular in bioinformatics.

In [ ]:
from sklearn.preprocessing import PowerTransformer

ptransform = PowerTransformer(standardize=False).fit(X_train)
print("Estimated lambda: ", ptransform.lambdas_)

X_train = ptransform.transform(X_train)
X_test = ptransform.transform(X_test)

It's ok to power transform all features, even almost normal. It usually won't hurt, but be sure to do a visual sanity check:

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(26, 14))
axes = axes.ravel()

X, Y = X_test, X_train
for ax, col in zip(axes, range(X.shape[1])):
  sns.histplot(x=X[:, col], ax=ax)
  ax.set_title(ddf.columns[col])

As expected, ~magic is forbidden outside of Hogwarts~ some variables are only approximately normal and there is little we can do about it.

The harsh reality is that there are many distributions that are far from normal or even lognormal. And that's okay, in most cases our algorithms are robust enough to handle this.


### Standardization

The second step is to bring the data to a similar scale. This is routinely done by a standartization, i.e. mean removal and variance scaling.

$$\hat{X} = \frac{X - E[X]}{\sigma}$$

After this transformation, the data should have zero mean and unit variance.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Check a random column:

In [ ]:
colind = 0
print(f"Train: {X_train[:, colind].mean():.3f}, {X_train[:, colind].std():.3f}")
print(f"Test: {X_test[:, colind].mean():.3f}, {X_test[:, colind].std():.3f}")

Visual check:

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(26, 14))
axes = axes.ravel()

X = X_test
for ax, col in zip(axes, range(X.shape[1])):
  sns.histplot(x=X[:, col], ax=ax)
  ax.set_title(ddf.columns[col])